### Example of expected stack/unstack behavior 

Here I outline using "stack" and "unstack" to associate a 1D "label" (e.g. labels generated by a classification algorithm) with each lat/lon pair in a dataset. 

Desired end result: an xarray Dataset where the "label" is associated with each lat/lon pair, with the coordinates and the association with other data variables intact. 

Problem: this works, but a similar approach is not working in my actual non-example code

## This code works

In [1]:
# import packages
import xarray as xr
import numpy as np

# generate random example data on a 2D "grid"
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2)
precipitation = 10 * np.random.rand(2, 2)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]

Next, we create a DataArray with the above data, dimensions "x" and "y", and lat/lon coordinates. The attributes are not necessary, but they are included for completeness. 

In [2]:
# create DataArray using the "temperature" data
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y"], temperature),
        precipitation=(["x", "y"], precipitation),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
    ),
    attrs=dict(description="Weather related data."),
)

ds

<xarray.Dataset>
Dimensions:        (x: 2, y: 2)
Coordinates:
    lon            (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat            (x, y) float64 42.25 42.21 42.63 42.59
Dimensions without coordinates: x, y
Data variables:
    temperature    (x, y) float64 29.11 18.2 22.83 32.93
    precipitation  (x, y) float64 4.237 6.459 4.376 8.918
Attributes:
    description:  Weather related data.

Next, we "stack" the DataArray, such that each lat/lon point is associated with a single value of a MultiIndex. This is necessary because of how my full code interacts with another package. 

In [3]:
# stack the DataArray
ds_stacked = ds.stack(z=("x","y"))
ds_stacked

<xarray.Dataset>
Dimensions:        (z: 4)
Coordinates:
    lon            (z) float64 -99.83 -99.32 -99.79 -99.23
    lat            (z) float64 42.25 42.21 42.63 42.59
  * z              (z) MultiIndex
  - x              (z) int64 0 0 1 1
  - y              (z) int64 0 1 0 1
Data variables:
    temperature    (z) float64 29.11 18.2 22.83 32.93
    precipitation  (z) float64 4.237 6.459 4.376 8.918
Attributes:
    description:  Weather related data.

Now, we generate a DataArray of labels. (Imagine that these have been generated by some kind of classification algorithm.)

In [4]:
# create 1D array of labels (to be associated with each lat/lon pair)
d1 = xr.DataArray(
    data=np.random.randint(low=0, high=2, size=(4,)),
    dims=["z"],
    attrs=dict(
        description="Label",
    ),
)

#d1 = d1.broadcast_like(ds.temperature)

Here, we combine the stacked DataArray with the "label" DataArray. 

In [5]:
# assign the labels in d1 to da_stacked
ds_stacked['label'] = d1
ds_stacked

<xarray.Dataset>
Dimensions:        (z: 4)
Coordinates:
  * z              (z) object (0, 0) (0, 1) (1, 0) (1, 1)
    lon            (z) float64 -99.83 -99.32 -99.79 -99.23
    lat            (z) float64 42.25 42.21 42.63 42.59
Data variables:
    temperature    (z) float64 29.11 18.2 22.83 32.93
    precipitation  (z) float64 4.237 6.459 4.376 8.918
    label          (z) int64 0 0 0 1
Attributes:
    description:  Weather related data.

In [6]:
# finally, unstack 
ds_final = ds_stacked.unstack()
ds_final 

<xarray.Dataset>
Dimensions:        (x: 2, y: 2)
Coordinates:
    lon            (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat            (x, y) float64 42.25 42.21 42.63 42.59
  * x              (x) int64 0 1
  * y              (y) int64 0 1
Data variables:
    temperature    (x, y) float64 29.11 18.2 22.83 32.93
    precipitation  (x, y) float64 4.237 6.459 4.376 8.918
    label          (x, y) int64 0 0 0 1
Attributes:
    description:  Weather related data.

NOTE: It just worked in my big code...but I don't know why. And now it's not working again. Augh! But I have seen that *it is possible!* The problem actually appears to come from the posterior probabilities. I think introducing the 'class' dimension messes things up. Let's only save the max and runner-up. 

Ooh! I appear to have reproduced the problem. When assigning the labels one, unstack still works. However, when assiging a real-valued one, unstack breaks! Is it about the values? Or something else? A coordinate problem? 

## Reproducing the problem

In [9]:
# generate random example data on a 2D "grid"
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2)
precipitation = 10 * np.random.rand(2, 2)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]

# create DataArray using the "temperature" data
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y"], temperature),
        precipitation=(["x", "y"], precipitation),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
    ),
    attrs=dict(description="Weather related data."),
)

ds

# stack the DataArray
ds_stacked = ds.stack(z=("x","y"))
ds_stacked

# create 1D array of labels (to be associated with each lat/lon pair)
d1 = xr.DataArray(
    data=np.random.randint(low=0, high=2, size=(4,)),
    dims=["z"],
    attrs=dict(
        description="Label",
    ),
)
d1

# create 1D array of labels (to be associated with each lat/lon pair)
d2 = xr.DataArray(
    data=np.random.randn(4,),
    dims=["z"],
    attrs=dict(
        description="Posteriors",
    ),
)
d2

# assign the labels in d1 to da_stacked
ds_stacked['label'] = d1
ds_stacked['post'] = d2
ds_stacked

# finally, unstack 
ds_final = ds_stacked.unstack()
ds_final 

<xarray.Dataset>
Dimensions:        (z: 4)
Coordinates:
  * z              (z) object (0, 0) (0, 1) (1, 0) (1, 1)
    lon            (z) float64 -99.83 -99.32 -99.79 -99.23
    lat            (z) float64 42.25 42.21 42.63 42.59
Data variables:
    temperature    (z) float64 29.11 18.2 22.83 32.93
    precipitation  (z) float64 4.237 6.459 4.376 8.918
    label          (z) int64 0 0 0 1
    post           (z) float64 0.144 1.454 0.761 0.1217
Attributes:
    description:  Weather related data.

### Trying Emma's broadcast solution

In [14]:
# generate random example data on a 2D "grid"
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2)
precipitation = 10 * np.random.rand(2, 2)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]

# create DataArray using the "temperature" data
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y"], temperature),
        precipitation=(["x", "y"], precipitation),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
    ),
    attrs=dict(description="Weather related data."),
)

ds

# stack the DataArray
ds_stacked = ds.stack(z=("x","y"))
ds_stacked

# create 1D array of labels (to be associated with each lat/lon pair)
d1 = xr.DataArray(
    data=np.random.randint(low=0, high=2, size=(4,)),
    dims=["z"],
    attrs=dict(
        description="Label",
    ),
)
d1

# create 1D array of labels (to be associated with each lat/lon pair)
d2 = xr.DataArray(
    data=np.random.randn(4,),
    dims=["z"],
    attrs=dict(
        description="Posteriors",
    ),
)
d2

# assign the labels in d1 to da_stacked
ds_stacked['label'] = d1
ds_stacked['post'] = d2
ds_stacked

# finally, unstack 
ds_final = ds_stacked.unstack()
ds_final 

<xarray.Dataset>
Dimensions:        (z: 4)
Coordinates:
  * z              (z) object (0, 0) (0, 1) (1, 0) (1, 1)
    lon            (z) float64 -99.83 -99.32 -99.79 -99.23
    lat            (z) float64 42.25 42.21 42.63 42.59
Data variables:
    temperature    (z) float64 29.11 18.2 22.83 32.93
    precipitation  (z) float64 4.237 6.459 4.376 8.918
    label          (z) int64 0 0 0 1
    post           (z) float64 0.144 1.454 0.761 0.1217
Attributes:
    description:  Weather related data.

In [16]:
d1.broadcast_like(ds.temperature)

<xarray.DataArray (x: 2, y: 2, z: 4)>
array([[[0, 0, 0, 1],
        [0, 0, 0, 1]],

       [[0, 0, 0, 1],
        [0, 0, 0, 1]]])
Dimensions without coordinates: x, y, z
Attributes:
    description:  Label

I'm not sure that this helps...